<a href="https://colab.research.google.com/github/chanita-boonkham/-ML-/blob/main/Week04-workshop-1/Lab04_NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import numpy as np #จัดการ metric vector
import pandas as pd
import matplotlib.pyplot as plt #กราฟ
import math

In [21]:
def accuracy_score(y_true, y_pred):

	"""	score = (y_true - y_pred) / len(y_true) """

	return round(float(sum(y_pred == y_true))/float(len(y_true)) * 100 ,2) #ทศนิมยม 2 ตน,นับจำนวนที่เหมือนกัน เป็นเศษส่วน แล้วคูณร้อย

In [22]:
y_pred = np.array([0,0,1,1,0,1])
y_true = np.array([0,0,0,0,0,1])
accuracy_score(y_true, y_pred)
print("accuracy score = ", accuracy_score(y_true, y_pred))

accuracy score =  66.67


In [23]:
def pre_processing(df):
  """ partioning data into features(X) and target(Y) """
  df = df.drop([df.columns[0]], axis = 1) #df = data frame, ตัด day ออก(drop)
  print(df.head())

  X = df.drop([df.columns[-1]], axis = 1) #-1 คือ ทางขวาสุด กลับหลัง , ตัดคอลัมน์สุดท้ายออก ไม่เอา yes no playtennis
  y = df[df.columns[-1]]
  return X, y


In [24]:
class  NaiveBayes: #class คือการเขียนโปรแกรมที่มีฟังก์ชันในตัวมัน

	"""
		Bayes Theorem:
										               Likelihood * Class prior probability
				Posterior Probability = -------------------------------------
											               Predictor prior probability

							  			          P(x|c) * p(c)
							   P(c|x) = ------------------
											                P(x)
	"""

	def __init__(self): # ในไพทอน __ คือ ฟังก์ชันของไพทอนเอง ไม่ได้สร้างมา

		"""
			Attributes:
				likelihoods: Likelihood of each feature per class
				class_priors: Prior probabilities of classes
				pred_priors: Prior probabilities of features
				features: All features of dataset
		"""
		self.features = list
		self.likelihoods = {} #dictionary json {'id','123'}
		self.class_priors = {}
		self.pred_priors = {} #evidence

		self.X_train = np.array
		self.y_train = np.array
		self.train_size = int
		self.num_feats = int

	def fit(self, X, y): #fit คือ train

		self.features = list(X.columns)
		self.X_train = X
		self.y_train = y
		self.train_size = X.shape[0]
		self.num_feats = X.shape[1]

		for feature in self.features: #จากแต่ละ feature
			self.likelihoods[feature] = {} #มี feature เก็บใน likelihood
			self.pred_priors[feature] = {}

			for feat_val in np.unique(self.X_train[feature]):
				self.pred_priors[feature].update({feat_val: 0})

				for outcome in np.unique(self.y_train):
					self.likelihoods[feature].update({feat_val+'_'+outcome:0})
					self.class_priors.update({outcome: 0})

		self._calc_class_prior()
		self._calc_likelihoods()
		self._calc_predictor_prior() #evidence

	def _calc_class_prior(self):

		""" P(c) - Prior Class Probability """

		for outcome in np.unique(self.y_train):
			outcome_count = sum(self.y_train == outcome)
			self.class_priors[outcome] = outcome_count / self.train_size

	def _calc_likelihoods(self):

		""" P(x|c) - Likelihood """

		for feature in self.features:

			for outcome in np.unique(self.y_train):
				outcome_count = sum(self.y_train == outcome)
				feat_likelihood = self.X_train[feature][self.y_train[self.y_train == outcome].index.values.tolist()].value_counts().to_dict()

				for feat_val, count in feat_likelihood.items():
					self.likelihoods[feature][feat_val + '_' + outcome] = count/outcome_count


	def _calc_predictor_prior(self):

		""" P(x) - Evidence """

		for feature in self.features:
			feat_vals = self.X_train[feature].value_counts().to_dict()

			for feat_val, count in feat_vals.items():
				self.pred_priors[feature][feat_val] = count/self.train_size


	def predict(self, X):

		""" Calculates Posterior probability P(c|x) """

		results = []
		X = np.array(X)

		for query in X:
			probs_outcome = {}
			for outcome in np.unique(self.y_train):
				prior = self.class_priors[outcome]
				likelihood = 1
				evidence = 1

				for feat, feat_val in zip(self.features, query):
					likelihood *= self.likelihoods[feat][feat_val + '_' + outcome]
					evidence *= self.pred_priors[feat][feat_val]

				posterior = (likelihood * prior) / (evidence)

				probs_outcome[outcome] = posterior

			result = max(probs_outcome, key = lambda x: probs_outcome[x])
			results.append(result)

		return np.array(results)




In [29]:
if __name__ == "__main__":

    #Weather Dataset
    print("\nWeather Dataset:")

    df = pd.read_csv("https://raw.githubusercontent.com/ekaratnida/Applied-machine-learning/master/Week10-desicion-tree/PlayTennis.csv")
    print(df.head())

    #Split fearures and target
    X,y  = pre_processing(df) #ในฟัันมีรีเทิน X,y เราก็ต้องไปรับอันนี้
    print("X.head()")
    print(X.head())
    print('***********************Begin Pre Processing************************')
    print("y.head()")
    print(y.head())

    nb_clf = NaiveBayes()
    nb_clf.fit(X, y)

    print("\nTrain Accuracy: {}".format(accuracy_score(y, nb_clf.predict(X))))

    #Query 1:
    query = np.array([['Rain','Mild', 'Normal', 'Strong']])
    print("Query 1:- {} ---> {}".format(query, nb_clf.predict(query)))

    #Query 2:
    query = np.array([['Overcast','Cool', 'Normal', 'Strong']])
    print("Query 2:- {} ---> {}".format(query, nb_clf.predict(query)))

    #Query 3:
    query = np.array([['Sunny','Hot', 'High', 'Strong']])
    print("Query 3:- {} ---> {}".format(query, nb_clf.predict(query)))


Weather Dataset:
  Day   Outlook Temperature Humidity    Wind Play_Tennis
0  D1     Sunny         Hot     High    Weak          No
1  D2     Sunny         Hot     High  Strong          No
2  D3  Overcast         Hot     High    Weak         Yes
3  D4      Rain        Mild     High    Weak         Yes
4  D5      Rain        Cool   Normal    Weak         Yes
    Outlook Temperature Humidity    Wind Play_Tennis
0     Sunny         Hot     High    Weak          No
1     Sunny         Hot     High  Strong          No
2  Overcast         Hot     High    Weak         Yes
3      Rain        Mild     High    Weak         Yes
4      Rain        Cool   Normal    Weak         Yes
X.head()
    Outlook Temperature Humidity    Wind
0     Sunny         Hot     High    Weak
1     Sunny         Hot     High  Strong
2  Overcast         Hot     High    Weak
3      Rain        Mild     High    Weak
4      Rain        Cool   Normal    Weak
***********************Begin Pre Processing************************